In [2]:
import pandas as pd 
import numpy as np
import re

import gensim
import en_core_web_sm
from gensim.models.phrases import Phrases, Phraser
from gensim.utils import simple_preprocess

########

import gensim
import en_core_web_sm

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
#########

import pickle

from time import time  # To time our operations

In [3]:
#Desktop
df = pd.read_csv('D:/Capstone/dataset/emails_v1.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209562 entries, 0 to 209561
Data columns (total 11 columns):
Unnamed: 0      209562 non-null int64
user            209562 non-null object
from            209562 non-null object
type            209562 non-null object
subject         209562 non-null object
date            209562 non-null object
bcc             209562 non-null object
email           209562 non-null object
body            209562 non-null object
word_count      209562 non-null int64
email_counts    209562 non-null int64
dtypes: int64(3), object(8)
memory usage: 17.6+ MB


In [5]:
#checkpoint
print(df.columns)
#print(df1.columns,'\n')
df=df.drop(columns = ['Unnamed: 0'])

print('number of rows:',df.shape[0])
print('number of bbc:',df[df.bcc!='None'].shape[0])
print('number of sent users:',df[df.type=='sent']['user'].value_counts().shape[0])

Index(['Unnamed: 0', 'user', 'from', 'type', 'subject', 'date', 'bcc', 'email',
       'body', 'word_count', 'email_counts'],
      dtype='object')
number of rows: 209562
number of bbc: 123
number of sent users: 147


In [6]:
nlp = en_core_web_sm.load(disable=['parser'])

In [7]:
custom_stop_words = ['from', 'subject', 're', 'edu', 'use','fwd','please',
                   'thanks','enron','inc','copy','mail','know','need',
                   're','yo','said','also','email','www','may','cell','fax',
                   'http','br','table','width','href','ahref','align','would',
                   'font','go','width_height','tr','nbsp','face_arial','gif','the',
                    'align_center', 'td_td','table_width','colspan_nbsp','colspan_align',
                    'like','courier_nbsp','ueta_ucita','hmtf']

for w in custom_stop_words:
    nlp.vocab[w].is_stop = True

In [8]:
# tokenize - break down each sentence into a list of words
def sent_to_words(df):
    list_of_sentences = df.values.tolist()
    for sentence in list_of_sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [9]:
data_words = list(sent_to_words(df.body))

In [10]:
bigram_mod = Phraser(Phrases(data_words, min_count=5, threshold=40))

In [11]:
def stop2lem(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
        
    tokens = [[word for word in simple_preprocess(str(doc))] for doc in texts]
    bigrams = [bigram_mod[doc] for doc in tokens]
    #no_stop = [token for token in bigrams if not token.is_stop]
    
    for sent in bigrams:
        doc = nlp(" ".join(sent))
        no_stop = [token for token in doc if not token.is_stop]
        texts_out.append([token.lemma_ for token in no_stop if token.pos_ in allowed_postags])
    return texts_out

In [12]:
t=time()
data_lemmatized = stop2lem(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print('Time to tokenize: {} mins'.format(round((time() - t) / 60, 2)))

Time to tokenize: 50.74 mins


In [13]:
pickle_out= open('spacy_lemv2.pickle','wb')
pickle.dump(data_lemmatized,pickle_out)
pickle_out.close()

In [6]:
pickle_in = open("spacy_lemv2.pickle","rb")
spacy_text = pickle.load(pickle_in)